# Segment Anything Keras Core port Benchmarks

This notebook benchmarks the segment anything model for TensorFlow, JAX, and PyTorch using Keras Core.

There are three types of benchmarks:

1. End-to-end model inference (`image_encoder + prompt_encoder + mask_decoder`)
2. End-to-end model inference with pre and post-processing
3. Prompt benchmarks (`prompt_encoder + mask_decoder` with image features set)

## Get all the dependencies and weight sets

In [1]:
# Get the dependencies
!pip install -q git+https://github.com/keras-team/keras-cv.git >> /dev/null
!pip install -q git+https://github.com/tirthasheshpatel/segment_anything_keras.git >> /dev/null
# Yank TensorFlow installation. For some reason, the default installation gives keras-nightly trouble
!pip list | grep tensorflow | awk '{print $1}' | xargs -n1 pip uninstall -y >> /dev/null
!pip uninstall -y keras >> /dev/null
!pip install -q keras-nightly tf-nightly tensorflow-datasets >> /dev/null
# Get the image for the demo
!curl -sSL https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/truck.jpg -o truck.jpg
!curl -sSL https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/groceries.jpg -o groceries.jpg

## Set the backend

In [1]:
import os
os.environ['KERAS_BACKEND'] = "torch"

## Choose the model

In [2]:
model_type = "huge"

## Import Dependencies

In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras_cv.backend import keras
from keras_cv.backend import ops
from keras_cv.models import SegmentAnythingModel
from sam_keras import SAMPredictor

## Define the model

In [4]:
sam = SegmentAnythingModel.from_preset(f"sam_{model_type}_sa1b")

## End-to-End Model Inference with pre and post-processing

### Setup

In [5]:
# Define predictor
model = SAMPredictor(sam)
transform  = model.transform

# Load the image
image = cv2.imread('truck.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Define the inputs
input_point = np.array([[500, 375]])
input_label = np.array([1])

image_record = {}

image_record["image"] = ops.convert_to_tensor(
    transform.apply_image(image)[np.newaxis, ...],
    dtype="float32"
)

image_record["original_size"] = (image.shape[0], image.shape[1])

image_record["point_coords"] = ops.reshape(
    ops.convert_to_tensor(
        input_point, dtype="float32"
    ),
    (1, 1, 2)
)
image_record["point_coords"] = transform.apply_coords(
    image_record["point_coords"], image_record["original_size"]
)

image_record["point_labels"] = ops.convert_to_tensor(
    input_label[np.newaxis, ...],
    dtype="float32"
)

### Benchmark

In [6]:
# Dry run to build the model
out = model.predict(image_record)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


In [7]:
# Predict also reports a time. Let's consider that too.
out = model.predict(image_record)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 604ms/step


In [8]:
# Benchmark the model
%timeit out = model.predict(image_record, verbose=0)

615 ms ± 2.87 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## End-to-End Model Inference

### Setup

In [9]:
# Run the pre and post-processing steps here itself
images = model.preprocess_images(image_record["image"])
points = ops.convert_to_tensor(
    image_record.get("point_coords", ops.ones((1, 0, 2))),
    dtype="float32"
)
labels = ops.convert_to_tensor(
    image_record.get("point_labels", ops.ones((1, 0))),
    dtype="float32"
)
box = ops.convert_to_tensor(
    image_record.get("boxes", ops.ones((1, 0, 2, 2))),
    dtype="float32"
)
mask = ops.convert_to_tensor(
    image_record.get("mask_inputs", ops.ones((1, 0, 256, 256, 1))),
    dtype="float32"
)

if ops.size(points) and not ops.size(box):
    pad_point = ops.zeros((points.shape[0], 1, 2), dtype="float32")
    pad_label = -ops.ones((labels.shape[0], 1), dtype="float32")
    points = ops.concatenate([points, pad_point], axis=1)
    labels = ops.concatenate([labels, pad_label], axis=1)

B = max([
    images.shape[0],
    points.shape[0],
    labels.shape[0],
    box.shape[0],
    mask.shape[0],
])

images, points, labels, box, mask = model._broadcast_batch(
    B, images, points, labels, box, mask
)

model_input = {
    "images": images,
    "points": points,
    "labels": labels,
    "boxes": box,
    "masks": mask
}

### Benchmark

In [10]:
model.model.predict(model_input);

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 613ms/step


In [11]:
%timeit model.model.predict(model_input, verbose=0)

616 ms ± 1.22 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Prompt Benchmarks

### Setup

In [12]:
# Set the features
features = ops.convert_to_tensor(
    model.model.backbone.predict(model_input["images"], verbose=0),
    dtype="float32"
)

In [13]:
class SAMPrompter(keras.Model):
    def __init__(self, prompt_encoder, mask_decoder, feature_shape=(64, 64, 256), **kwargs):
        # Define the prompt encoder inputs -- Prompts
        prompt_inputs = {
            "points": keras.Input(shape=[None, 2], name="points"),
            "labels": keras.Input(shape=[None], name="labels"),
            "boxes": keras.Input(shape=[None, 2, 2], name="boxes"),
            "masks": keras.Input(shape=[None, None, None, 1], name="masks"),
        }

        # All Inputs -- Features + Prompts
        all_inputs = {"features": keras.Input(feature_shape, name="features")}
        all_inputs.update(prompt_inputs)

        # Build the prompt encoder
        prompt_embeddings = prompt_encoder(prompt_inputs)

        # Define the mask decoder inputs
        mask_decoder_inputs = {
            "image_embeddings": all_inputs["features"],
            "image_pe": prompt_embeddings["dense_positional_embeddings"],
            "sparse_prompt_embeddings": prompt_embeddings["sparse_embeddings"],
            "dense_prompt_embeddings": prompt_embeddings["dense_embeddings"],
        }

        # Build the mask decoder
        outputs = mask_decoder(mask_decoder_inputs)

        super().__init__(inputs=all_inputs, outputs=outputs, **kwargs)

        self.prompt_encoder = prompt_encoder
        self.mask_decoder = mask_decoder

In [14]:
prompter_model = SAMPrompter(model.model.prompt_encoder, model.model.mask_decoder, feature_shape=features.shape[1:])

In [15]:
prompt_inputs = {
    "features": features,
    "points": model_input["points"],
    "labels": model_input["labels"],
    "boxes": model_input["boxes"],
    "masks": model_input["masks"]
}

### Benchmark

In [16]:
# Dry run to build the model
outs = prompter_model.predict(prompt_inputs)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


In [17]:
# Predict also reports a time. Let's also consider that.
outs = prompter_model.predict(prompt_inputs)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


In [18]:
%timeit outs = prompter_model.predict(prompt_inputs, verbose=0)

39.6 ms ± 1.94 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
